# nuScenes devkit tutorial

Welcome to the nuScenes tutorial. This demo assumes the database itself is available at `./data/`, and loads a mini version of the full dataset.

## A Gentle Introduction to nuScenes

In this part of the tutorial, let us go through a top-down introduction of our database. Our dataset comprises of elemental building blocks that are the following:

1. `log` - Log information from which the data was extracted.
2. `scene` - 20 second snippet of a car's journey.
3. `sample` - An annotated snapshot of a scene at a particular timestamp.
4. `sample_data` - Data collected from a particular sensor.
5. `ego_pose` - Ego vehicle poses at a particular timestamp.
6. `sensor` - A specific sensor type.
7. `calibrated sensor` - Definition of a particular sensor as calibrated on a particular vehicle.
8. `instance` - Enumeration of all object instance we observed.
9. `category` - Taxonomy of object categories (e.g. vehicle, human). 
10. `attribute` - Property of an instance that can change while the category remains the same.
11. `visibility` - Fraction of pixels visible in all the images collected from 6 different cameras.
12. `sample_annotation` - An annotated instance of an object within our interest.
13. `map` - Map data that is stored as binary semantic masks from a top-down view.

The database schema is visualized below. For more information see the [nuScenes schema](https://github.com/nutonomy/nuscenes-devkit/blob/master/docs/schema_nuscenes.md) page.
![](https://www.nuscenes.org/public/images/nuscenes-schema.svg)

## Initialization

In [1]:
import sys

sys.path.append('./build')

from _nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='./data/', verbose=True)

Loading NuScenes dataset for version v1.0-mini
Loaded 23 category.
Loaded 8 attribute.
Loaded 4 visibility.
Loaded 911 instance.
Loaded 12 sensor.
Loaded 120 calibrated_sensor.
Loaded 31206 ego_pose.
Loaded 8 log.
Loaded 10 scene.
Loaded 404 sample.
Loaded 31206 sample_data.
Loaded 18538 sample_annotation.
Loaded 4 map.
Loaded data in 592 ms
Built reverse index in 31 ms


## A look at the dataset

### 1. `scene`

nuScenes is a large scale database that features annotated samples across ***1000 scenes*** of approximately 20 seconds each. Let's take a look at the scenes that we have in the loaded database.

In [2]:
nusc.scenes

[Scene {
   token: cc8c0bf57f984915a77078b10eb33198
   nbr_samples: 39
   first_sample_token: ca9a282c9e77460f8360f564131a8af5
   last_sample_token: ed5fc18c31904f96a8f0dbb99ff069c0
   name: scene-0061
 },
 Scene {
   token: fcbccedd61424f1b85dcbf8f897f9754
   nbr_samples: 40
   first_sample_token: 3e8750f331d7499e9b5123e9eb70f2e2
   last_sample_token: 281b92269fd648d4b52d06ac06ca6d65
   name: scene-0103
 },
 Scene {
   token: 6f83169d067343658251f72e1dd17dbc
   nbr_samples: 41
   first_sample_token: 8687ba92abd3406aa797115b874ebeba
   last_sample_token: dcbe451d383e450786aaad04ab9d3790
   name: scene-0553
 },
 Scene {
   token: bebf5f5b2a674631ab5c88fd1aa9e87a
   nbr_samples: 41
   first_sample_token: 5991fad3280c4f84b331536c32001a04
   last_sample_token: 35833ae5808e4ef186d1fdebac3d9cf6
   name: scene-0655
 },
 Scene {
   token: 2fc3753772e241f2ab2cd16a784cc680
   nbr_samples: 41
   first_sample_token: cd9964f8c3d34383b16e9c2997de1ed0
   last_sample_token: 8fe9664cec514a58b1184c4fcef

Let's look at a scene metadata

In [3]:
my_scene = nusc.scenes[0]
my_scene

Scene {
  token: cc8c0bf57f984915a77078b10eb33198
  nbr_samples: 39
  first_sample_token: ca9a282c9e77460f8360f564131a8af5
  last_sample_token: ed5fc18c31904f96a8f0dbb99ff069c0
  name: scene-0061
}

### 2. `sample`

In scenes, we annotate our data every half a second (2 Hz).

We define `sample` as an ***annotated keyframe of a scene at a given timestamp***. A keyframe is a frame where the time-stamps of data from all the sensors should be very close to the time-stamp of the sample it points to.

Now, let us look at the first annotated sample in this scene.

In [4]:
first_sample_token = my_scene.first_sample_token
first_sample_token

'ca9a282c9e77460f8360f564131a8af5'

Let's examine its metadata

In [5]:
my_sample = nusc.sample(first_sample_token)
my_sample

Sample {
  token: ca9a282c9e77460f8360f564131a8af5
  timestamp: 1532402927647951
  scene_token: cc8c0bf57f984915a77078b10eb33198
}

A useful method is  `list_sample()` which lists all related `sample_data` keyframes and `sample_annotation` associated with a `sample` which we will discuss in detail in the subsequent parts.

In [6]:
print(len(my_sample.annotations))
print(len(my_sample.datas))

69
12


### 3. `sample_data`

The nuScenes dataset contains data that is collected from a full sensor suite. Hence, for each snapshot of a scene, we provide references to a family of data that is collected from these sensors. 

We provide a `data` key to access these:

In [7]:
my_sample.datas

[SampleData {
   token: 37091c75b9704e0daa829ba56dfa0906
   sample_token: ca9a282c9e77460f8360f564131a8af5
   filename: samples/RADAR_FRONT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT__1532402927664178.pcd
   ego_pose_token: 37091c75b9704e0daa829ba56dfa0906
   calibrated_sensor_token: f4d2a6c281f34a7eb8bb033d82321f79
 },
 SampleData {
   token: 11946c1461d14016a322916157da3c7d
   sample_token: ca9a282c9e77460f8360f564131a8af5
   filename: samples/RADAR_FRONT_LEFT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT_LEFT__1532402927652686.pcd
   ego_pose_token: 11946c1461d14016a322916157da3c7d
   calibrated_sensor_token: 2ee327ac0903407dbb42c754861c1e63
 },
 SampleData {
   token: 491209956ee3435a9ec173dad3aaf58b
   sample_token: ca9a282c9e77460f8360f564131a8af5
   filename: samples/RADAR_FRONT_RIGHT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT_RIGHT__1532402927639817.pcd
   ego_pose_token: 491209956ee3435a9ec173dad3aaf58b
   calibrated_sensor_token: 341ad78c836f4dc4bd2c61ad12285a63
 },
 SampleData 

### 4. `sample_annotation`

`sample_annotation` refers to any ***bounding box defining the position of an object seen in a sample***. All location data is given with respect to the global coordinate system. Let's examine an example from our `sample` above.

In [8]:
my_annotation_token = my_sample.annotations[18].token
my_annotation_metadata =  nusc.annotation(my_annotation_token)
my_annotation_metadata

Annotation {
  token: 83d881a6b3d94ef3a3bc3b585cc514f8
  sample_token: ca9a282c9e77460f8360f564131a8af5
  instance_token: e91afa15647c4c4994f19aeb302c7179
  visibility_token: 4
  attribute_tokens: [  58aa28b1c2a54dc88e169808c07331e3,   ]
  translation:   Translation {
    [409.989000, 1164.099000, 1.623000,     ]
  }
  rotation:   Rotation {
    [-0.582882, 0.000000, 0.000000, 0.812557,     ]
  }
  size: [2.877000, 10.201000, 3.595000,   ]
  prev_token: 
  next_token: f3721bdfd7ee4fd2a4f94874286df471
}

### 5. `instance`

Object instance are instances that need to be detected or tracked by an AV (e.g a particular vehicle, pedestrian). Let us examine an instance metadata

In [9]:
my_instance = nusc.instances[599]
my_instance

Instance {
  token: 9cba9cd8af85487fb010652c90d845b5
  category_token: fedb11688db84088883945752e480c2c
  nbr_annotations: 16
}

We generally track an instance across different frames in a particular scene. However, we do not track them across different scenes. In this example, we have 16 annotated samples for this instance across a particular scene.

In [10]:
instance_token = my_instance.token

### 6. `category`

A `category` is the object assignment of an annotation.  Let's look at the category table we have in our database. The table contains the taxonomy of different object categories and also list the subcategories (delineated by a period). 

In [11]:
nusc.categories

[Category {
   token: 1fa93b757fc74fb197cdd60001ad8abf
   name: human.pedestrian.adult
   description: Adult subcategory.
 },
 Category {
   token: b1c6de4c57f14a5383d9f963fbdcb5cb
   name: human.pedestrian.child
   description: Child subcategory.
 },
 Category {
   token: b2d7c6c701254928a9e4d6aac9446d79
   name: human.pedestrian.wheelchair
   description: Wheelchairs. If a person is in the wheelchair, include in the annotation.
 },
 Category {
   token: 6a5888777ca14867a8aee3fe539b56c4
   name: human.pedestrian.stroller
   description: Strollers. If a person is in the stroller, include in the annotation.
 },
 Category {
   token: 403fede16c88426885dd73366f16c34a
   name: human.pedestrian.personal_mobility
   description: A small electric or self-propelled vehicle, e.g. skateboard, segway, or scooters, on which the person typically travels in a upright position. Driver and (if applicable) rider should be included in the bounding box along with the vehicle.
 },
 Category {
   token: bb

A category record contains the name and the description of that particular category.

In [12]:
nusc.categories[9]

Category {
  token: dfd26f200ade4d24b540184e16050022
  name: vehicle.motorcycle
  description: Gasoline or electric powered 2-wheeled vehicle designed to move rapidly (at the speed of standard cars) on the road surface. This category includes all motorcycles, vespas and scooters.
}

Refer to `instructions_nuscenes.md` for the definitions of the different categories.

### 7. `attribute`

An `attribute` is a property of an instance that may change throughout different parts of a scene while the category remains the same. Here we list the provided attributes and the number of annotations associated with a particular attribute.

In [13]:
nusc.attributes

[Attribute {
   token: cb5118da1ab342aa947717dc53544259
   name: vehicle.moving
   description: Vehicle is moving.
 },
 Attribute {
   token: c3246a1e22a14fcb878aa61e69ae3329
   name: vehicle.stopped
   description: Vehicle, with a driver/rider in/on it, is currently stationary but has an intent to move.
 },
 Attribute {
   token: 58aa28b1c2a54dc88e169808c07331e3
   name: vehicle.parked
   description: Vehicle is stationary (usually for longer duration) with no immediate intent to move.
 },
 Attribute {
   token: a14936d865eb4216b396adae8cb3939c
   name: cycle.with_rider
   description: There is a rider on the bicycle or motorcycle.
 },
 Attribute {
   token: 5a655f9751944309a277276b8f473452
   name: cycle.without_rider
   description: There is NO rider on the bicycle or motorcycle.
 },
 Attribute {
   token: 03aa62109bf043afafdea7d875dd4f43
   name: pedestrian.sitting_lying_down
   description: The human is sitting or lying down.
 },
 Attribute {
   token: 4d8821270b4a47e3a8a300cbec48

Let's take a look at an example how an attribute may change over one scene

In [14]:
my_instance = nusc.instances[27]
first_token = my_instance.first_annotation_token
last_token = my_instance.last_annotation_token
nbr_samples = my_instance.nbr_annotations
current_token = first_token

found_change = False
last_attr = None
for ann in my_instance.annotations:
    cur_attr_token = ann.attribute_tokens[0]
    cur_attr = nusc.attribute(cur_attr_token).name
    if cur_attr != last_attr:
        print("Changed from `{}` to `{}` at timestamp {} out of {} annotated timestamps".format(last_attr, cur_attr, ann.sample.timestamp, nbr_samples))
        found_change = True
    last_attr = cur_attr


Changed from `None` to `pedestrian.moving` at timestamp 1532402927647951 out of 39 annotated timestamps
Changed from `pedestrian.moving` to `pedestrian.standing` at timestamp 1532402938197897 out of 39 annotated timestamps


### 8. `visibility`

`visibility` is defined as the fraction of pixels of a particular annotation that are visible over the 6 camera feeds, grouped into 4 bins.

In [15]:
nusc.visibilities

[Visibility {
   token: 1
   name: v0-40
   description: visibility of whole object is between 0 and 40%
 },
 Visibility {
   token: 2
   name: v40-60
   description: visibility of whole object is between 40 and 60%
 },
 Visibility {
   token: 3
   name: v60-80
   description: visibility of whole object is between 60 and 80%
 },
 Visibility {
   token: 4
   name: v80-100
   description: visibility of whole object is between 80 and 100%
 }]

Let's look at an example `sample_annotation` with 80-100% visibility

In [16]:
anntoken = 'a7d0722bce164f88adf03ada491ea0ba'
visibility = nusc.annotation(anntoken).visibility_token

print(f"Visibility: {visibility}")

Visibility: 4


Let's look at an example `sample_annotation` with 0-40% visibility

In [17]:
anntoken = '9f450bf6b7454551bbbc9a4c6e74ef2e'
visibility_token = nusc.annotation(anntoken).visibility_token
visibility = nusc.visibility(visibility_token).token

print(f"Visibility: {visibility}")

Visibility: 1


### 9. `sensor`

The nuScenes dataset consists of data collected from our full sensor suite which consists of:
- 1 x LIDAR, 
- 5 x RADAR, 
- 6 x cameras, 

In [18]:
nusc.sensors

[Sensor {
   token: 725903f5b62f56118f4094b46a4470d8
   channel: CAM_FRONT
   modality: camera
   channel: CAM_FRONT
   channel: CAM_FRONT
 },
 Sensor {
   token: ce89d4f3050b5892b33b3d328c5e82a3
   channel: CAM_BACK
   modality: camera
   channel: CAM_BACK
   channel: CAM_BACK
 },
 Sensor {
   token: a89643a5de885c6486df2232dc954da2
   channel: CAM_BACK_LEFT
   modality: camera
   channel: CAM_BACK_LEFT
   channel: CAM_BACK_LEFT
 },
 Sensor {
   token: ec4b5d41840a509984f7ec36419d4c09
   channel: CAM_FRONT_LEFT
   modality: camera
   channel: CAM_FRONT_LEFT
   channel: CAM_FRONT_LEFT
 },
 Sensor {
   token: 2f7ad058f1ac5557bf321c7543758f43
   channel: CAM_FRONT_RIGHT
   modality: camera
   channel: CAM_FRONT_RIGHT
   channel: CAM_FRONT_RIGHT
 },
 Sensor {
   token: ca7dba2ec9f95951bbe67246f7f2c3f7
   channel: CAM_BACK_RIGHT
   modality: camera
   channel: CAM_BACK_RIGHT
   channel: CAM_BACK_RIGHT
 },
 Sensor {
   token: dc8b396651c05aedbb9cdaae573bb567
   channel: LIDAR_TOP
   modalit

Every `sample_data` has a record on which `sensor` the data is collected from (note the "channel" key)

In [19]:
nusc.sample_datas[10]

SampleData {
  token: 2ecfec536d984fb491098c9db1404117
  sample_token: 356d81f38dd9473ba590f39e266f54e5
  filename: sweeps/RADAR_FRONT/n015-2018-07-24-11-22-45+0800__RADAR_FRONT__1532402928269133.pcd
  ego_pose_token: 2ecfec536d984fb491098c9db1404117
  calibrated_sensor_token: f4d2a6c281f34a7eb8bb033d82321f79
}

### 10. `calibrated_sensor`

`calibrated_sensor` consists of the definition of a particular sensor (lidar/radar/camera) as calibrated on a particular vehicle. Let us look at an example.

In [20]:
nusc.calibrated_sensors[0]

CalibratedSensor {
  token: f4d2a6c281f34a7eb8bb033d82321f79
  sensor_token: 47fcd48f71d75e0da5c8c1704a9bfe0a
}

Note that the `translation` and the `rotation` parameters are given with respect to the ego vehicle body frame. 

### 11. `ego_pose`

`ego_pose` contains information about the location (encoded in `translation`) and the orientation (encoded in `rotation`) of the ego vehicle, with respect to the global coordinate system.

In [21]:
nusc.ego_positions[0]

EgoPosition {
  token: 5ace90b379af485b9dcb1584b01e7212
  timestamp: 1532402927814384
  translation:   Translation {
    [410.778786, 1179.467329, 0.000000,     ]
  }
  rotation:   Rotation {
    [0.573179, -0.001581, 0.013859, -0.819312,     ]
  }
}

Note that the number of `ego_pose` records in our loaded database is the same as the number of `sample_data` records. These two records exhibit a one-to-one correspondence.

### 12. `log`

The `log` table contains log information from which the data was extracted. A `log` record corresponds to one journey of our ego vehicle along a predefined route. Let's check the number of logs and the metadata of a log.

In [22]:
print("Number of `logs` in our loaded database: {}".format(len(nusc.logs)))

Number of `logs` in our loaded database: 8


In [23]:
nusc.logs[0]

Log {
  token: 7e25a2c8ea1f41c5b0da1e69ecfa71a2
  logfile: n015-2018-07-24-11-22-45+0800
  vehicle: n015
  date_captured: 2018-07-24
}

Notice that it contains a variety of information such as the date and location of the log collected. It also gives out information about the map from where the data was collected. Note that one log can contain multiple non-overlapping scenes.

### 13. `map`

Map information is stored as binary semantic masks from a top-down view. Let's check the number of maps and metadata of a map.

In [24]:
print("There are {} maps masks in the loaded dataset".format(len(nusc.maps)))

There are 4 maps masks in the loaded dataset


In [25]:
nusc.maps[0]

Map {
  token: 53992ee3023e5494b90c316c183be829
  filename: maps/53992ee3023e5494b90c316c183be829.png
}

## nuScenes Basics

Let's get a bit technical.

The NuScenes class holds several tables. Each table is a list of records, and each record is a dictionary. For example the first record of the category table is stored at:

In [26]:
nusc.categories[0]

Category {
  token: 1fa93b757fc74fb197cdd60001ad8abf
  name: human.pedestrian.adult
  description: Adult subcategory.
}

The category table is simple: it holds the fields `name` and `description`. It also has a `token` field, which is a unique record identifier. Since the record is a dictionary, the token can be accessed like so:

In [27]:
cat_token = nusc.categories[0].token
cat_token

'1fa93b757fc74fb197cdd60001ad8abf'

If you know the `token` for any record in the DB you can retrieve the record by doing

In [28]:
nusc.category(cat_token)

Category {
  token: 1fa93b757fc74fb197cdd60001ad8abf
  name: human.pedestrian.adult
  description: Adult subcategory.
}

_As you can notice, we have recovered the same record!_

OK, that was easy. Let's try something harder. Let's look at the `sample_annotation` table.

In [29]:
nusc.annotations[0]

Annotation {
  token: 70aecbe9b64f4722ab3c230391a3beb8
  sample_token: cd21dbfc3bd749c7b10a5c42562e0c42
  instance_token: 6dd2cbf4c24b4caeb625035869bca7b5
  visibility_token: 4
  attribute_tokens: [  4d8821270b4a47e3a8a300cbec48188e,   ]
  translation:   Translation {
    [373.214000, 1130.480000, 1.250000,     ]
  }
  rotation:   Rotation {
    [0.983110, 0.000000, 0.000000, -0.183016,     ]
  }
  size: [0.621000, 0.669000, 1.642000,   ]
  prev_token: a1721876c0944cdd92ebc3c75d55d693
  next_token: 1e8e35d365a441a18dd5503a0ee1c208
}

This also has a `token` field (they all do). In addition, it has several fields of the format [a-z]*\_token, _e.g._ instance_token. These are foreign keys in database terminology, meaning they point to another table. 
Using `nusc.get()` we can grab any of these in constant time. For example, let's look at the visibility record.

In [30]:
nusc.visibility(nusc.annotations[0].visibility_token)

Visibility {
  token: 4
  name: v80-100
  description: visibility of whole object is between 80 and 100%
}

The visibility records indicate how much of an object was visible when it was annotated.

Let's also grab the `instance_token`

In [31]:
one_instance = nusc.instance(nusc.annotations[0].instance_token)
one_instance

Instance {
  token: 6dd2cbf4c24b4caeb625035869bca7b5
  category_token: 1fa93b757fc74fb197cdd60001ad8abf
  nbr_annotations: 39
}

This points to the `instance` table. This table enumerate the object _instances_ we have encountered in each 
scene. This way we can connect all annotations of a particular object.

If you look carefully at the README tables, you will see that the sample_annotation table points to the instance table, 
but the instance table doesn't list all annotations that point to it. 

So how can we recover all sample_annotations for a particular object instance? There are two ways:

1. Directly use `instance.annotations`. Let's try it:

In [32]:
anns = one_instance.annotations
anns

[Annotation {
   token: ef63a697930c4b20a6b9791f423351da
   sample_token: ca9a282c9e77460f8360f564131a8af5
   instance_token: 6dd2cbf4c24b4caeb625035869bca7b5
   visibility_token: 1
   attribute_tokens: [  4d8821270b4a47e3a8a300cbec48188e,   ]
   translation:   Translation {
     [373.256000, 1130.419000, 0.800000,     ]
   }
   rotation:   Rotation {
     [0.983110, 0.000000, 0.000000, -0.183016,     ]
   }
   size: [0.621000, 0.669000, 1.642000,   ]
   prev_token: 
   next_token: 7987617983634b119e383d8a29607fd7
 },
 Annotation {
   token: 7987617983634b119e383d8a29607fd7
   sample_token: 39586f9d59004284a7114a68825e8eec
   instance_token: 6dd2cbf4c24b4caeb625035869bca7b5
   visibility_token: 1
   attribute_tokens: [  4d8821270b4a47e3a8a300cbec48188e,   ]
   translation:   Translation {
     [373.256000, 1130.419000, 0.810000,     ]
   }
   rotation:   Rotation {
     [0.983110, 0.000000, 0.000000, -0.183016,     ]
   }
   size: [0.621000, 0.669000, 1.642000,   ]
   prev_token: ef63a

This returns a list of all sample_annotation records with the `'instance_token'` == `one_instance['token']`. Let's store these in a set for now

In [33]:
ann_tokens_field2token = set([a.token for a in anns])
ann_tokens_field2token

{'05e0ad1194804f548be544f2267c7e74',
 '060be69422ee4b2a8b239b463b995e92',
 '146b2111cc0c401ca09d96777758d81e',
 '163d85048698495dbf55a35f613c5fb9',
 '17a6843fefcd4b2b811eddbb1ccd708d',
 '1de9ad564050444486eb587360cf135f',
 '1e8e35d365a441a18dd5503a0ee1c208',
 '23f10d5f0d254068941be8797493c7eb',
 '2b5948828cdb49e3be6be1320381bbbf',
 '3a930d1793434d9a8a87d6eba28ff70e',
 '3b24f083c0bf42d695a1040efdab7ffe',
 '4e41d9560dbf46cab1568b8ef6a282f3',
 '6f371d3f0d7d494eaa6f81daa3df58c0',
 '70aecbe9b64f4722ab3c230391a3beb8',
 '74f550e3257c4f52af1102c0d49d37b8',
 '7670ac8bc5044d5a9e11e205c839385d',
 '794fcc425f074a1392206ed925fdbbd8',
 '7987617983634b119e383d8a29607fd7',
 '7fa3a688931b4500b7ce29d187d3b975',
 '807b3e029a6b4e428f6cc82fc26a35a7',
 '8bb63134d48840aaa2993f490855ff0d',
 '90d94112b9ea4fb691e988b40af5b161',
 '913072e56d6c4025b9b47ba085dd6d7c',
 '93d5b79041c64693a5b32f1103a39a06',
 '9acb7dfed3454f72b2874dda3bdacc48',
 'a1721876c0944cdd92ebc3c75d55d693',
 'a2b20cdbf1ed4018ac795b8845d5deaa',
 

The `nusc.field2token()` method is generic and can be used in any similar situation.

2. For certain situation, we provide some reverse indices in the tables themselves. This is one such example. 

The instance record has a field `first_annotation_token` which points to the first annotation in time of this instance. 
Recovering this record is easy.

In [34]:
ann_record = one_instance.annotations[0]
ann_record

Annotation {
  token: ef63a697930c4b20a6b9791f423351da
  sample_token: ca9a282c9e77460f8360f564131a8af5
  instance_token: 6dd2cbf4c24b4caeb625035869bca7b5
  visibility_token: 1
  attribute_tokens: [  4d8821270b4a47e3a8a300cbec48188e,   ]
  translation:   Translation {
    [373.256000, 1130.419000, 0.800000,     ]
  }
  rotation:   Rotation {
    [0.983110, 0.000000, 0.000000, -0.183016,     ]
  }
  size: [0.621000, 0.669000, 1.642000,   ]
  prev_token: 
  next_token: 7987617983634b119e383d8a29607fd7
}

Now we can traverse all annotations of this instance using the "next" field. Let's try it. 

In [35]:
ann_tokens_traverse = set()
ann_tokens_traverse.add(ann_record.token)
while not ann_record.next_token == "":
    ann_record = nusc.annotation(ann_record.next_token)
    ann_tokens_traverse.add(ann_record.token)

Finally, let's assert that we recovered the same ann_records as we did using nusc.field2token:

In [36]:
print(len(ann_tokens_field2token))
print(len(ann_tokens_traverse))
print(ann_tokens_traverse == ann_tokens_field2token)

39
39
True


## Reverse indexing and short-cuts

The nuScenes tables are normalized, meaning that each piece of information is only given once.
For example, there is one `map` record for each `log` record. Looking at the schema you will notice that the `map` table has a `log_token` field, but that the `log` table does not have a corresponding `map_token` field. But there are plenty of situations where you have a `log`, and want to find the corresponding `map`! So what to do? You can always use the `nusc.field2token()` method, but that is slow and inconvenient. We therefore add reverse mappings for some common situations including this one.

Further, there are situations where one needs to go through several tables to get a certain piece of information. 
Consider, for example, the category name (e.g. `human.pedestrian`) of a `sample_annotation`. The `sample_annotation` table doesn't hold this information since the category is an instance level constant. Instead the `sample_annotation` table points to a record in the `instance` table. This, in turn, points to a record in the `category` table, where finally the `name` fields stores the required information.

Since it is quite common to want to know the category name of an annotation, we add a `category_name` field to the `sample_annotation` table during initialization of the NuScenes class.

In this section, we list the short-cuts and reverse indices that are added to the `NuScenes` class during initialization. These are all created in the `NuScenes.__make_reverse_index__()` method.

### Reverse indices
We add two reverse indices by default.
* A `map_token` field is added to the `log` records.
* The `sample` records have shortcuts to all `sample_annotations` for that record as well as `sample_data` key-frames. Confer `nusc.list_sample()` method in the previous section for more details on this.

## Data Visualizations

We provide list and rendering methods. These are meant both as convenience methods during development, and as tutorials for building your own visualization methods. They are implemented in the NuScenesExplorer class, with shortcuts through the NuScenes class itself.

### List methods
There are three list methods available.

1. `list_categories()` lists all categories, counts and statistics of width/length/height in meters and aspect ratio.

In [37]:
nusc.categories

[Category {
   token: 1fa93b757fc74fb197cdd60001ad8abf
   name: human.pedestrian.adult
   description: Adult subcategory.
 },
 Category {
   token: b1c6de4c57f14a5383d9f963fbdcb5cb
   name: human.pedestrian.child
   description: Child subcategory.
 },
 Category {
   token: b2d7c6c701254928a9e4d6aac9446d79
   name: human.pedestrian.wheelchair
   description: Wheelchairs. If a person is in the wheelchair, include in the annotation.
 },
 Category {
   token: 6a5888777ca14867a8aee3fe539b56c4
   name: human.pedestrian.stroller
   description: Strollers. If a person is in the stroller, include in the annotation.
 },
 Category {
   token: 403fede16c88426885dd73366f16c34a
   name: human.pedestrian.personal_mobility
   description: A small electric or self-propelled vehicle, e.g. skateboard, segway, or scooters, on which the person typically travels in a upright position. Driver and (if applicable) rider should be included in the bounding box along with the vehicle.
 },
 Category {
   token: bb

2. `list_attributes()` lists all attributes and counts.

In [38]:
nusc.attributes

[Attribute {
   token: cb5118da1ab342aa947717dc53544259
   name: vehicle.moving
   description: Vehicle is moving.
 },
 Attribute {
   token: c3246a1e22a14fcb878aa61e69ae3329
   name: vehicle.stopped
   description: Vehicle, with a driver/rider in/on it, is currently stationary but has an intent to move.
 },
 Attribute {
   token: 58aa28b1c2a54dc88e169808c07331e3
   name: vehicle.parked
   description: Vehicle is stationary (usually for longer duration) with no immediate intent to move.
 },
 Attribute {
   token: a14936d865eb4216b396adae8cb3939c
   name: cycle.with_rider
   description: There is a rider on the bicycle or motorcycle.
 },
 Attribute {
   token: 5a655f9751944309a277276b8f473452
   name: cycle.without_rider
   description: There is NO rider on the bicycle or motorcycle.
 },
 Attribute {
   token: 03aa62109bf043afafdea7d875dd4f43
   name: pedestrian.sitting_lying_down
   description: The human is sitting or lying down.
 },
 Attribute {
   token: 4d8821270b4a47e3a8a300cbec48

3. `list_scenes()` lists all scenes in the loaded DB.

In [39]:
nusc.scenes

[Scene {
   token: cc8c0bf57f984915a77078b10eb33198
   nbr_samples: 39
   first_sample_token: ca9a282c9e77460f8360f564131a8af5
   last_sample_token: ed5fc18c31904f96a8f0dbb99ff069c0
   name: scene-0061
 },
 Scene {
   token: fcbccedd61424f1b85dcbf8f897f9754
   nbr_samples: 40
   first_sample_token: 3e8750f331d7499e9b5123e9eb70f2e2
   last_sample_token: 281b92269fd648d4b52d06ac06ca6d65
   name: scene-0103
 },
 Scene {
   token: 6f83169d067343658251f72e1dd17dbc
   nbr_samples: 41
   first_sample_token: 8687ba92abd3406aa797115b874ebeba
   last_sample_token: dcbe451d383e450786aaad04ab9d3790
   name: scene-0553
 },
 Scene {
   token: bebf5f5b2a674631ab5c88fd1aa9e87a
   nbr_samples: 41
   first_sample_token: 5991fad3280c4f84b331536c32001a04
   last_sample_token: 35833ae5808e4ef186d1fdebac3d9cf6
   name: scene-0655
 },
 Scene {
   token: 2fc3753772e241f2ab2cd16a784cc680
   nbr_samples: 41
   first_sample_token: cd9964f8c3d34383b16e9c2997de1ed0
   last_sample_token: 8fe9664cec514a58b1184c4fcef